# Analyze results for 3D images
Jan 21, 2021

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_3d_image_analysis import *

# sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
# import post_analysis_pandas as post

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [5]:
img_size=64
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}


### Read validation data and compute metrics

In [6]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'64':raw_data_dir+'3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'}

In [7]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
s_val=f_transform(s_val)
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

# del(s_val)

(1000, 64, 64, 64)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/modules_3d_image_analysis.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


### Read run data

In [8]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [9]:
u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)

In [77]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210330_210607_3d_l2.0_80k_Dold_Gnew_models', '2021…

In [78]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210210_060657_3d_l0.5_80k


## Plot Losses

In [79]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [80]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
49991,49991.0,19.0,0.139178,0.153671,0.292849,3.335879,109.390038,106.054161,0.593765,107.882362,1.828320,3.040272,-3.639444,-3.297790,0.174207
49992,49992.0,19.0,0.151336,0.157718,0.309054,3.575520,115.043243,111.467728,0.449586,114.440796,1.527064,3.897165,-3.095383,-3.544273,0.176674
49993,49993.0,19.0,0.105302,0.151941,0.257243,4.033356,110.972748,106.939392,0.099652,111.905945,0.928245,3.616718,-3.611984,-4.010709,0.176776
49994,49994.0,19.0,0.161123,0.149665,0.310788,3.421758,112.690910,109.269150,-0.230819,108.171524,1.120545,3.100078,-4.062793,-3.382043,0.170807
49995,49995.0,19.0,0.125806,0.125898,0.251705,3.648645,112.789368,109.140724,-0.082451,110.255234,0.878076,3.464474,-3.210974,-3.614910,0.166331
49996,49996.0,19.0,0.142615,0.127887,0.270503,3.645512,112.513634,108.868118,-0.282605,108.630768,0.782956,3.549009,-3.601730,-3.614101,0.166848
49997,49997.0,19.0,0.130128,0.139888,0.270016,3.888697,113.607445,109.718750,-0.130014,107.163147,0.519461,4.171101,-3.464474,-3.864824,0.167543
49998,49998.0,19.0,0.142242,0.135834,0.278076,3.917274,108.965904,105.048630,-0.458831,109.337723,0.388810,3.689531,-3.848390,-3.890647,0.166817
49999,49999.0,19.0,0.112726,0.145006,0.257732,3.998548,112.899139,108.900589,-0.236680,110.021439,-0.211256,3.616997,-3.077670,-3.973144,0.167481
50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.670441,0.003983,NaN,NaN,NaN,NaN


In [81]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [82]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-1.3191087806224824


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
48746,48746.0,19.0,0.221948,0.163785,0.385733,3.410356,114.700104,111.289749,-0.073481,111.955978,-2.790811,2.865679,-4.862306,-3.368082,0.168443
48573,48573.0,19.0,0.125482,0.176527,0.302009,5.168800,114.598160,109.429359,-0.358928,110.427399,-2.744849,4.403152,-2.852283,-5.159686,0.178840
41438,41438.0,16.0,0.161493,0.144947,0.306440,3.483413,115.430450,111.947037,0.689315,111.940392,-2.720657,4.068726,-3.346916,-3.443439,0.168607
431,431.0,0.0,0.228316,0.148959,0.377275,3.119493,114.276299,111.156807,-2.459732,111.252029,-2.701512,6.020895,-3.991484,-3.074159,0.165140
432,432.0,0.0,0.249108,0.363809,0.612917,6.504613,117.375618,110.871002,-2.476121,110.937500,-2.689993,5.678924,-0.919672,-6.503074,0.165737
433,433.0,0.0,0.244808,0.158577,0.403385,6.625327,117.526093,110.900764,-2.516761,110.103714,-2.658638,4.899019,-4.433754,-6.623944,0.165499
430,430.0,0.0,0.153840,0.233067,0.386907,6.291269,117.471123,111.179855,-2.358588,111.141769,-2.600845,5.256094,-7.552483,-6.289396,0.165749
429,429.0,0.0,0.232126,0.349030,0.581156,9.688788,120.807060,111.118271,-2.235708,111.092361,-2.400405,5.397780,-11.089845,-9.688726,0.165266
47666,47666.0,19.0,0.154022,0.171831,0.325853,3.498048,115.066818,111.568771,-0.182188,111.938362,-2.375428,2.960806,-4.767941,-3.462143,0.169265
48650,48650.0,19.0,0.148364,0.128720,0.277085,3.328581,112.363953,109.035370,-0.034871,111.167877,-2.361971,3.332144,-3.328040,-3.280420,0.166201


In [83]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [84]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [85]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [86]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [87]:
df_sliced=w.result
# df_sliced

In [88]:
best_step=[]
df_test=df.copy()
# df_test=df[df.epoch>5]
best_step.append(f_slice_df(df_test,cutoff=0.9,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.8515629876156938, 'chi_spec1': 17.629881713761986}
[0, 19]
{'chi_1': 0.4467854450791812, 'chi_spec1': 12.326062544700253}
[0, 19]
{'chi_1': 0.20935169432385645, 'chi_spec1': 7.148777479045893}
[array([44900, 33790, 40160, 28200]), array([ 6560, 29210]), array([40250, 41080])]
[ 6560 28200 29210 33790 40160 40250 41080 44900]


In [89]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [90]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 22)
[(2, 6560), (11, 28200), (11, 29210), (13, 33790), (16, 40160), (16, 40250), (16, 41080), (17, 44900)]


In [91]:
df_best[col_list]


,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
655,2-6560,0.246885,10.027608,49.063214,0.192645,0.053953,0.000288,0.115025,3.904391,17956.069610,2,6560
2819,11-28200,0.036677,6.014820,49.570766,0.019308,0.014379,0.002990,0.010212,9.884761,1447.230356,11,28200
2920,11-29210,0.248065,6.946046,59.081433,0.241302,0.006456,0.000306,0.089805,33.648975,50.338523,11,29210
3378,13-33790,0.032535,6.317830,50.120442,0.028484,0.002187,0.001864,0.013548,6.696044,1461.858703,13,33790
4015,16-40160,0.032709,5.377288,50.452657,0.027667,0.003498,0.001544,0.013563,10.932140,240.525028,16,40160
4024,16-40250,0.100561,3.085800,43.490086,0.071879,0.025700,0.002983,0.051874,7.510610,71.322752,16,40250
4107,16-41080,0.159360,3.133683,47.506667,0.103007,0.053081,0.003272,0.063929,15.402410,45.373778,16,41080
4489,17-44900,0.023849,4.554960,53.793261,0.017225,0.005775,0.000849,0.004996,17.831019,80.620578,17,44900


### Plots 2: stored data

In [92]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=64
    assert plot_type in ['hist','spec','grid','spec_relative','hist_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative','hist_relative']:     fig=plt.figure(figsize=(6,6))


    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)


        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            
            
        if plot_type=='hist_relative':

            x2=row.hist_bin_centers
            y2=row.hist_val
            yerr2=row.hist_err
            x2=f_invtransform(x2)
            
#             plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['hist_val'],dict_bkg['hist_err']

            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            plt.xlim(0,img_size/2)
            plt.ylim(0,5)
            plt.legend()

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:,0]
            print(images.shape)
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.xlim(0,img_size/2)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)
        plt.legend()
        
    if plot_type=='hist_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [93]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative','hist_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('2-6560', '11-28200', '11-29210', '13…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [94]:
print(s_val.shape)


(1000, 64, 64, 64)


In [95]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210123_050147_3d_l0.5_80k/images/gen_img_epoch-28_step-71620.npy'
a1=np.load(fname)

In [96]:
s_val.shape,a1.shape

((1000, 64, 64, 64), (32, 64, 64, 64))

In [97]:
f_plot_grid(a1[6:14,:,:,0],cols=4,fig_size=(4,2))

2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
f_plot_grid(s_val[100:108,:,:,0],cols=4,fig_size=(4,2))

2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
655,2-6560,0.246885,10.027608,49.063214,0.192645,0.053953,0.000288,0.115025,3.904391,17956.069610,2,6560
2819,11-28200,0.036677,6.014820,49.570766,0.019308,0.014379,0.002990,0.010212,9.884761,1447.230356,11,28200
2920,11-29210,0.248065,6.946046,59.081433,0.241302,0.006456,0.000306,0.089805,33.648975,50.338523,11,29210
3378,13-33790,0.032535,6.317830,50.120442,0.028484,0.002187,0.001864,0.013548,6.696044,1461.858703,13,33790
4015,16-40160,0.032709,5.377288,50.452657,0.027667,0.003498,0.001544,0.013563,10.932140,240.525028,16,40160
4024,16-40250,0.100561,3.085800,43.490086,0.071879,0.025700,0.002983,0.051874,7.510610,71.322752,16,40250
4107,16-41080,0.159360,3.133683,47.506667,0.103007,0.053081,0.003272,0.063929,15.402410,45.373778,16,41080
4489,17-44900,0.023849,4.554960,53.793261,0.017225,0.005775,0.000849,0.004996,17.831019,80.620578,17,44900


In [100]:
dims = [2,3,4,7]
np_x = np.random.uniform(size=dims).astype(np.float32)